In [1]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py

In [2]:
from huggingface_hub import HfFileSystem
from tqdm import tqdm
import mp
import os
import json

fs = HfFileSystem()

In [3]:
# files = fs.ls("datasets/malaysia-ai/pretrain-text-dataset/")

In [4]:
# selected = []
# for f in files:
#     if not f['name'].endswith('.jsonl'):
#         continue
#     if f['size'] >= 1e+9:
#         continue   
#     selected.append(f)
    
# len(selected)

In [5]:
# def loop(files):
#     files, _ = files
#     for f in tqdm(files):
#         if os.path.exists(os.path.split(f['name'])[1]):
#             continue
#         fs.download(f['name'], './')

In [6]:
# mp.multiprocessing(selected, loop, cores = 10, returned = False)

In [7]:
# files = fs.ls("datasets/malaysia-ai/crawl-astroawani")
# files = [f for f in files if f['name'].endswith('v2.json.nested')]
# for f in files:
#     fs.download(f['name'], './')

In [8]:
from glob import glob

alls = []
for f in tqdm(glob('*.jsonl')):
    selected = []
    with open(f) as fopen:
        for l in fopen:
            try:
                l = json.loads(l)
            except:
                continue
            if not isinstance(l, str):
                continue
            if 200 < len(l.split()) < 3072:
                selected.append(l)
            
            if len(selected) > 700:
                alls.extend(selected)
                break
                
len(alls)

100%|█████████████████████████████████████████| 283/283 [00:21<00:00, 13.41it/s]


107253

In [9]:
from glob import glob

for f in tqdm(glob('*-v2.json.nested')):
    selected = []
    with open(f) as fopen:
        d = json.load(fopen)
    for d_ in d:
        try:
            l = d_['r']['response']['articleBody']
        except:
            continue
        if not isinstance(l, str):
            continue
        if 200 < len(l.split()) < 3072:
            selected.append(l)
            
        if len(selected) > 3000:
            alls.extend(selected)
            break
        
len(alls)

100%|█████████████████████████████████████████████| 8/8 [00:02<00:00,  3.86it/s]


128260

In [10]:
prompts = []
for a in alls:
    p = f"""
paragraph = ```
{a}
```

generate 100 words persona based on paragraph above, example generated persona like below,

```
Zara is a strong and patient woman but still has emotional problems due to her unhappy divorce relationship. However, she remains optimistic and persistent in maintaining her dignity and wants to be an example for other women who face the same challenges. She also likes to seek advice and support from her friends.
```
Or,
```
Azmi bin Ahmad is a strong and persistent individual in computer science. However, he also faces emotional challenges due to his unending divorce relationships. Despite this, Azmi remains optimistic and determined to maintain his dignity and hopes to be an example for other individuals facing similar challenges.
```
Or,
```
Panglima Awang, also known as Enrique of Malacca, is a brave and wise Malay hero. As a war captive who was taken from Melaka to Lisbon, Portuguese, he had struggled with strength and intelligence to survive on foreign lands. Enrique possessed an unassuming spirit and outstanding language skills, enabling him to communicate with various nations.
```
    """
    prompts.append((p.strip(), a))

In [11]:
# !rm -rf generate-persona
!mkdir generate-persona

mkdir: cannot create directory ‘generate-persona’: File exists


In [12]:
import requests
import os

def answer(q, i):
    filename = os.path.join('generate-persona', f'{i}.json')
    
    if os.path.exists(filename) and os.path.getsize(filename) > 200:
        return
    
    for _ in range(3):
        try:
            headers = {
                'accept': 'application/json',
                'Content-Type': 'application/json',
            }
            
            messages = [
                {'role': 'user', 'content': q[0]}
            ]

            json_data = {
                'model': 'mallam-small-2.5',
                'messages': messages,
                'temperature': 0.9,
                'top_p': 0.8,
                'top_k': 50,
                'max_tokens': 4096,
            }

            response = requests.post('http://localhost:8000/v1/chat/completions', 
                                     headers=headers, json=json_data)
            r = response.json()
            r = r['choices'][0]['message']['content']
            if len(r):
                with open(filename, 'w') as fopen:
                    json.dump({'r': r, 'context': q[1]}, fopen)
            
            return
        except:
            pass

In [13]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [14]:
from threading import Thread
from queue import Queue

queue = Queue()
urls = [(q, no) for no, q in enumerate(prompts)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [15]:
answer(*urls[0])

In [16]:
answer(*urls[1])

In [17]:
answer(*urls[-1])

In [ ]:
from tqdm import tqdm

max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

 82%|█████████████████████████▌     | 105674/128260 [3:33:47<2:18:56,  2.71it/s]